# Aspect Based Sentiment Analysis

## Prepare Data

In [57]:
import pandas as pd
import numpy as np
import spacy
from rake_nltk import Rake

In [6]:
path = '/Users/cblanesg/Downloads/Womens Clothing E-Commerce Reviews.csv'
raw = pd.read_csv(path)

In [7]:
dataset = raw[['Review Text', 'Rating','Class Name' ]]

In [8]:
dataset

,Review Text,Rating,Class Name
0,Absolutely wonderful - silky and sexy and comf...,4,Intimates
1,Love this dress! it's sooo pretty. i happene...,5,Dresses
2,I had such high hopes for this dress and reall...,3,Dresses
3,"I love, love, love this jumpsuit. it's fun, fl...",5,Pants
4,This shirt is very flattering to all due to th...,5,Blouses
...,...,...,...
23481,I was very happy to snag this dress at such a ...,5,Dresses
23482,"It reminds me of maternity clothes. soft, stre...",3,Knits
23483,"This fit well, but the top was very see throug...",3,Dresses
23484,I bought this dress for a wedding i have this ...,3,Dresses


In [9]:
def label_sentiment(row):
    if row['Rating'] < 3 :
        return 'Negative'
    if row['Rating'] == 3: 
        return 'Neutral'
    if row['Rating'] > 3 :
        return 'Positive'
    return 'Other'

In [10]:
dataset['Sentiment'] = dataset.apply( lambda Rating: label_sentiment(Rating), axis = 1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
dataset_clean = dataset[['Sentiment', 'Class Name', 'Review Text']]

In [12]:
dataset_clean = dataset_clean.rename(columns = {'Sentiment':'sentiment', 
                               'Class Name': 'aspect_category',
                               'Review Text': 'review'})

In [13]:
dataset_clean.review = dataset_clean.review.str.lower()

In [45]:
num_item = list(range(len(list_reviews)))

Next step is to obtain **Aspect Terms**. 
Approaches that don't require training data:
1. Word Frequency
2. Word Co-occurrences
3. TF-IDF (Frequency inverse document frequency)
4. RAKE (Rapid Automatic Keyword Extraction)

In [71]:
r = Rake()

['absolutely wonderful', 'silky', 'sexy', 'comfortable']

In [ ]:
list_reviews = dataset_clean.review.tolist()

In [75]:
letters_only = []
for item in list_reviews:
    letters_only.append(re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(item)))

In [80]:
aspect_terms = []
for item in letters_only:
    r.extract_keywords_from_text(item)
    terms = r.get_ranked_phrases() 
    aspect_terms.append(aspect_terms)

In [87]:
for item in letters_only:
    aspect_terms = []
    r.extract_keywords_from_text(item)
    terms = r.get_ranked_phrases() 
    aspect_terms.append(terms)

In [90]:
def get_keywords(text):
    r.extract_keywords_from_text(text)
    terms = r.get_ranked_phrases() 
    return(terms)

In [94]:
all_terms = []
for i in letters_only:
    all_terms.append(get_keywords(i))

In [134]:
aspect_terms = []
for i in all_terms:
    aspect_terms.append(' '.join(i))

# Co-occurrence Matrix

In [136]:
# Generate co ocurrence matrix

In [137]:
aspect_terms

['absolutely wonderful silky sexy comfortable',
 'knee would definitely never would true midi sooo pretty truly petite online bc petite bc store someone ordered love little length hits happened glad find dress bought',
 'several somewhat cheap net major design flaw layer sewn directly usual size top half really wanted petite medium ok overall layers imo initially ordered high hopes fit nicely bottom half petite small outrageously small net layer small zipper zip work tight reordered found fact dress could comfortable c',
 'love love love fabulous every time great compliments get nothing fun flirty wear jumpsuit',
 'adjustable front tie perfect length pairs well cardigan love wear sleeveless shirt leggings flattering due',
 'skirt would take away love tracy reese dresses usually wear small frame feet tall alterations shortening skirt love work style stranger returned pretty petite package p overwhelmed one narrowing lot long idea garment full embellishment dress color brand',
 'hte msal

In [166]:
corpus = ' '.join(aspect_terms)

In [171]:
vocab = set(corpus.split())
token_sent_list = [word_tokenize(sen) for sen in aspect_terms]

In [174]:
from nltk.tokenize import word_tokenize
from itertools import combinations
from collections import Counter

co_occ = {ii:Counter({jj:0 for jj in vocab if jj!=ii}) for ii in vocab}
k=2

for sen in token_sent_list:
    for ii in range(len(sen)):
        if ii < k:
            c = Counter(sen[0:ii+k+1])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c
        elif ii > len(sen)-(k+1):
            c = Counter(sen[ii-k::])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c
        else:
            c = Counter(sen[ii-k:ii+k+1])
            del c[sen[ii]]
            co_occ[sen[ii]] = co_occ[sen[ii]] + c
co_occ = {ii:dict(co_occ[ii]) for ii in vocab}
display(co_occ)

KeyError: 'can'

In [124]:
aspect_terms = pd.DataFrame(aspect_terms)

In [103]:
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [104]:
aspect_categories_model = Sequential()
aspect_categories_model.add(Dense(512, input_shape=(6000,), activation='relu'))
aspect_categories_model.add(Dense(12, activation='softmax'))
aspect_categories_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [154]:
from keras.preprocessing.text import Tokenizer

vocab_size = 6000 # We set a maximum size for the vocabulary
tokenizer = Tokenizer(num_words=vocab_size)

In [165]:
tokenizer.fit_on_texts(dataset_clean.review[0])

In [162]:
if type(dataset_clean.review) != str:
else:
    tokenizer.fit_on_texts(dataset_clean.review)

SyntaxError: 'continue' not properly in loop (<ipython-input-162-a9bbf5ef542f>, line 5)

In [157]:
aspect_tokenized = pd.DataFrame(tokenizer.texts_to_matrix(aspect_terms))